In [18]:
from simple_coin_machine import SimpleCoinMachine
from my_states import LockedState, UnlockedState, BrokenState
import my_states

device = SimpleCoinMachine()

DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


In [2]:
device.on_event('pass')

In [3]:
device.on_event('pin_entered')

This event pin_entered is not supported in this state
Supported events are ['coin', 'failed']


In [4]:
device.on_event('coin')

current collected_amount is 0
please enter the amount of coins20


DEBUG:root:Entered UnlockedState


In [5]:
device.on_event('coin')

Thanks!


In [6]:
print (device.state)
device.on_event('failed')

DEBUG:root:Entered BrokenState


UnlockedState
Machine is Out of order


In [7]:
device.on_event('fixed')

DEBUG:root:Entered LockedState


In [8]:
device.on_event('coin')

current collected_amount is 0
please enter the amount of coins20


DEBUG:root:Entered UnlockedState


In [9]:
device.on_event('fixed')

This event fixed is not supported in this state
Supported events are ['pass', 'coin', 'failed']


### Unit Tests:
- Follwing cell shows the unit tests
- Please **run the tests via command line**, as the notebook is not handling command line params as expected
- To run the tests via command line: **python -m unittest test**

In [62]:
# test.py

import unittest
import io
import sys
from contextlib import contextmanager

from simple_coin_machine import SimpleCoinMachine
import my_states

@contextmanager
def replace_stdin(target):
    orig = sys.stdin
    sys.stdin = target
    yield
    sys.stdin = orig


class TestNotebook(unittest.TestCase):

    def setUp(self):
        self.device = SimpleCoinMachine()

    def test_state_initial(self): # tests machine starts in Locked state with collected_amount = 0
        self.assertEqual(str(self.device.state), "LockedState")
        self.assertEqual(self.device.state.collected_amount, 0)

    # test (a subset of) state transitions which are not allowed

    def test_illegal_moves_LockedState(self):
        self.assertEqual(str(self.device.state), "LockedState")
        self.device.on_event('pass')
        self.assertEqual(str(self.device.state), "LockedState")
        self.device.on_event('fixed')
        self.assertEqual(str(self.device.state), "LockedState")

        with replace_stdin(io.StringIO("10")): # with 10 coins machine stays in LockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "LockedState")
        
    def test_illegal_moves_UnlockedState(self):
        self.assertEqual(str(self.device.state), "LockedState")
        
        with replace_stdin(io.StringIO("20")): # with 20 coins machine moves to UnlockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "UnlockedState")
            
        self.device.on_event('fixed')
        self.assertEqual(str(self.device.state), "UnlockedState")

    def test_illegal_moves_BrokenState(self):
        self.assertEqual(str(self.device.state), "LockedState")
        self.device.on_event('failed')
        self.assertEqual(str(self.device.state), "BrokenState")
    
    def test_accumulate(self):
        with replace_stdin(io.StringIO("10")): # with 10 coins machine stays in LockedState
            self.device.on_event('coin')
            self.assertEqual(self.device.state.collected_amount, 10)
            self.assertEqual(str(self.device.state), "LockedState")
            
        with replace_stdin(io.StringIO("20")): # with 20 coins machine moves to UnlockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "UnlockedState")        

    # test (a subset of) state transitions which are allowed
    
    def test_legal_moves_LockedState(self):
        self.assertEqual(str(self.device.state), "LockedState")
        self.device.on_event('failed')
        self.assertEqual(str(self.device.state), "BrokenState")

        self.device.on_event('fixed') # move back to LockedState
        with replace_stdin(io.StringIO("10")): # with 10 coins machine stays in LockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "LockedState")

        with replace_stdin(io.StringIO("20")): # with 20 coins machine moves to UnlockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "UnlockedState")

    def test_legal_moves_BrokenState(self): 
        self.device.on_event('failed')
        self.assertEqual(str(self.device.state), "BrokenState")

        self.device.on_event('fixed') # move back to LockedState
        self.assertEqual(str(self.device.state), "LockedState")

    def test_legal_moves_UnlockedState(self):
        with replace_stdin(io.StringIO("20")): # with 20 coins machine goes in UnlockedState
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "UnlockedState")
            self.device.on_event('failed')
            self.assertEqual(str(self.device.state), "BrokenState")
            self.device.on_event('fixed') # move back to LockedState

        with replace_stdin(io.StringIO("20")):    
            self.device.on_event('coin')
            self.assertEqual(str(self.device.state), "UnlockedState")
            self.device.on_event('pass')
            self.assertEqual(str(self.device.state), "LockedState")


unittest.main(argv=[''], verbosity=2, exit=False)

test_accumulate (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


current collected_amount is 0
please enter the amount of coins10
Still need 10 coins to unlock
current collected_amount is 10
please enter the amount of coins20


DEBUG:root:Entered UnlockedState
ok
test_illegal_moves_BrokenState (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState
DEBUG:root:Entered BrokenState
ok
test_illegal_moves_LockedState (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


Machine is Out of order
This event pass is not supported in this state
Supported events are ['coin', 'failed']
This event fixed is not supported in this state
Supported events are ['coin', 'failed']
current collected_amount is 0
please enter the amount of coins10


ok
test_illegal_moves_UnlockedState (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


...

DEBUG:root:Entered UnlockedState
ok
test_legal_moves_UnlockedState (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


current collected_amount is 0
please enter the amount of coins20


DEBUG:root:Entered UnlockedState
DEBUG:root:Entered BrokenState
DEBUG:root:Entered LockedState


Machine is Out of order
current collected_amount is 0
please enter the amount of coins20


DEBUG:root:Entered UnlockedState
DEBUG:root:Entered LockedState
ok
test_state_initial (__main__.TestNotebook) ... DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState
ok

----------------------------------------------------------------------
Ran 8 tests in 98.794s

OK
